# Ames Housing Notebook: 04 - Kaggle Submission
***
### Importing basline packages as well as scikit learn - linear_model, model_selection, metrics, and pre-processing packages

In [190]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

%matplotlib inline

### Reading in the test csv from the datasets folder

In [249]:
train_df = pd.read_csv('../datasets//train_clean.csv')

In [191]:
test_df = pd.read_csv('../datasets/test.csv')

In [192]:
test_df.head(2)

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD


### Setting index to be Id and fixing column names to be lower case / snake_case

In [193]:
test_df = test_df.set_index('Id')
# change names
test_df.columns = [column_name.lower().replace(' ', '_') for column_name in test_df.columns]
# only numeric columns
#numeric_cols = test_df.describe().columns
#test_df = test_df[numeric_cols]

### Assessing null values in each column

In [194]:
test_df.isnull().sum().sort_values(ascending = False).head(20)

pool_qc           874
misc_feature      837
alley             820
fence             706
fireplace_qu      422
lot_frontage      160
garage_cond        45
garage_qual        45
garage_yr_blt      45
garage_finish      45
garage_type        44
bsmtfin_type_1     25
bsmt_exposure      25
bsmt_qual          25
bsmt_cond          25
bsmtfin_type_2     25
mas_vnr_area        1
electrical          1
mas_vnr_type        1
year_built          0
dtype: int64

### Dropping columns with the most null values

In [195]:
test_df = test_df.drop(['pool_qc', 'misc_feature', 'alley', 'fence', 'fireplace_qu','lot_frontage'], axis=1)

In [196]:
test_df.isnull().sum().sort_values(ascending = False).head(15)

garage_cond       45
garage_qual       45
garage_yr_blt     45
garage_finish     45
garage_type       44
bsmtfin_type_2    25
bsmt_qual         25
bsmt_cond         25
bsmt_exposure     25
bsmtfin_type_1    25
electrical         1
mas_vnr_area       1
mas_vnr_type       1
foundation         0
exter_cond         0
dtype: int64

### Filling in the remainder of null columns with median values

In [197]:
# train_df2 = 
test_df = test_df.fillna(test_df.median())

In [198]:
test_df.isnull().sum().sort_values(ascending = False).head(15)

garage_cond       45
garage_finish     45
garage_qual       45
garage_type       44
bsmtfin_type_1    25
bsmt_exposure     25
bsmt_cond         25
bsmt_qual         25
bsmtfin_type_2    25
electrical         1
mas_vnr_type       1
roof_style         0
roof_matl          0
exterior_1st       0
sale_type          0
dtype: int64

### None of the remainder of null columns were filled with median values, which means that their data types are objects
***
#### Performing get_dummies method on test_df and merging dummy columns with test_df columns

In [199]:
dummy_test = pd.get_dummies(test_df)

In [200]:
test_df = pd.merge(test_df, dummy_test, right_index=True, left_index=True)

In [201]:
test_df.isnull().sum().sort_values(ascending = False).head(12)

garage_cond       45
garage_qual       45
garage_finish     45
garage_type       44
bsmt_qual         25
bsmt_cond         25
bsmt_exposure     25
bsmtfin_type_1    25
bsmtfin_type_2    25
electrical         1
mas_vnr_type       1
sale_type_WD       0
dtype: int64

### Filling in the rest of the columns with null values one by one with the options that have the highest value count

In [202]:
test_df[['garage_cond']].groupby(test_df['garage_cond']).agg(['count', 'size', 'nunique'])

garage_cond             
                  count size nunique
garage_cond                         
Ex                    1    1       1
Fa                   27   27       1
Gd                    3    3       1
Po                    6    6       1
TA                  796  796       1

In [203]:
test_df['garage_cond'].fillna('TA', inplace = True)

In [204]:
test_df[['garage_qual']].groupby(test_df['garage_qual']).agg(['count', 'size', 'nunique'])

garage_qual             
                  count size nunique
garage_qual                         
Fa                   42   42       1
Gd                    6    6       1
Po                    3    3       1
TA                  782  782       1

In [205]:
test_df['garage_qual'].fillna('TA', inplace = True)

In [206]:
test_df[['garage_finish']].groupby(test_df['garage_finish']).agg(['count', 'size', 'nunique'])

garage_finish             
                      count size nunique
garage_finish                           
Fin                     218  218       1
RFn                     233  233       1
Unf                     382  382       1

In [207]:
test_df['garage_finish'].fillna('Unf', inplace = True)

In [208]:
test_df[['garage_type']].groupby(test_df['garage_type']).agg(['count', 'size', 'nunique'])

garage_type             
                  count size nunique
garage_type                         
2Types                4    4       1
Attchd              518  518       1
Basment               9    9       1
BuiltIn              53   53       1
CarPort               4    4       1
Detchd              246  246       1

In [209]:
test_df['garage_type'].fillna('Attchd', inplace = True)

In [210]:
test_df[['bsmt_qual']].groupby(test_df['bsmt_qual']).agg(['count', 'size', 'nunique'])

bsmt_qual             
              count size nunique
bsmt_qual                       
Ex               73   73       1
Fa               28   28       1
Gd              355  355       1
Po                1    1       1
TA              396  396       1

In [211]:
test_df['bsmt_qual'].fillna('TA', inplace = True)

In [212]:
test_df[['bsmt_cond']].groupby(test_df['bsmt_cond']).agg(['count', 'size', 'nunique'])

bsmt_cond             
              count size nunique
bsmt_cond                       
Fa               39   39       1
Gd               33   33       1
TA              781  781       1

In [213]:
test_df['bsmt_cond'].fillna('TA', inplace = True)

In [214]:
test_df[['bsmt_exposure']].groupby(test_df['bsmt_exposure']).agg(['count', 'size', 'nunique'])

bsmt_exposure             
                      count size nunique
bsmt_exposure                           
Av                      130  130       1
Gd                       80   80       1
Mn                       76   76       1
No                      567  567       1

In [215]:
test_df['bsmt_exposure'].fillna('No', inplace = True)

In [216]:
test_df[['bsmtfin_type_1']].groupby(test_df['bsmtfin_type_1']).agg(['count', 'size', 'nunique'])

bsmtfin_type_1             
                        count size nunique
bsmtfin_type_1                            
ALQ                       136  136       1
BLQ                        69   69       1
GLQ                       243  243       1
LwQ                        52   52       1
Rec                       105  105       1
Unf                       248  248       1

In [217]:
test_df['bsmtfin_type_1'].fillna('Unf', inplace = True)

In [218]:
test_df[['bsmtfin_type_2']].groupby(test_df['bsmtfin_type_2']).agg(['count', 'size', 'nunique'])

bsmtfin_type_2             
                        count size nunique
bsmtfin_type_2                            
ALQ                        18   18       1
BLQ                        20   20       1
GLQ                        11   11       1
LwQ                        29   29       1
Rec                        26   26       1
Unf                       749  749       1

In [219]:
test_df['bsmtfin_type_2'].fillna('Unf', inplace = True)

In [220]:
test_df[['electrical']].groupby(test_df['electrical']).agg(['count', 'size', 'nunique'])

electrical             
                count size nunique
electrical                        
FuseA              48   48       1
FuseF              15   15       1
FuseP               1    1       1
SBrkr             813  813       1

In [221]:
test_df['electrical'].fillna('SBrkr', inplace = True)

In [222]:
test_df[['mas_vnr_type']].groupby(test_df['mas_vnr_type']).agg(['count', 'size', 'nunique'])

mas_vnr_type             
                    count size nunique
mas_vnr_type                          
BrkCmn                 12   12       1
BrkFace               250  250       1
CBlock                  1    1       1
None                  534  534       1
Stone                  80   80       1

In [223]:
test_df['mas_vnr_type'].fillna('None', inplace = True)

In [224]:
test_df.isnull().sum().sort_values(ascending = False).head(12)

sale_type_WD         0
land_contour_Bnk     0
3ssn_porch_y         0
screen_porch_y       0
pool_area_y          0
misc_val_y           0
mo_sold_y            0
yr_sold_y            0
ms_zoning_C (all)    0
ms_zoning_FV         0
ms_zoning_I (all)    0
ms_zoning_RH         0
dtype: int64

### No more null values.  I will now prepare test_df with sale price predictions and put it in the proper format to be submitted to kaggle.

In [225]:
test_df.head(2)

,pid_x,ms_subclass_x,ms_zoning,lot_area_x,street,lot_shape,land_contour,utilities,lot_config,land_slope,...,sale_type_COD,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_VWD,sale_type_WD
Id,,,,,,,,,,,,,,,,,,,,,
2658,902301120,190,RM,9142,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,0,0,0,1
2718,905108090,90,RL,9662,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,0,0,0,1


### Creating X_test using the 24 variables I used to fit my MLR model to train_df

In [241]:
X_test = test_df[['overall_qual_x','gr_liv_area_x', 'garage_area_x', 'garage_cars_x', 
            'total_bsmt_sf_x', '1st_flr_sf_x', 'year_built_x', 'year_remod/add_x', 
            'full_bath_x', 'garage_yr_blt_x', 'mas_vnr_area_x', 'totrms_abvgrd_x', 
            'fireplaces_x', 'ms_subclass_x', 'open_porch_sf_x', 'wood_deck_sf_x', 
            'lot_area_x', 'bsmt_full_bath_x', 'half_bath_x', '2nd_flr_sf_x', 'bsmt_unf_sf_x',
            'bedroom_abvgr_x', 'screen_porch_x', 'overall_cond_x']] 

In [262]:
test_df['saleprice'] = 1

In [263]:
y = test_df['saleprice']

In [264]:
lr_kag = LinearRegression()
lr_kag.fit(X_test, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [265]:
y_preds_test = lr_kag.predict(X_test)

In [266]:
test_df['SalePrice'] = y_preds_test

In [267]:
submission_df = test_df[['SalePrice']]

In [269]:
submission_df

,SalePrice
Id,
2658,1.0
2718,1.0
2414,1.0
1989,1.0
625,1.0
...,...
1662,1.0
1234,1.0
1373,1.0
